In [5]:
#from crypt import methods
from distutils.command.config import config
from flask import Flask, flash, redirect, url_for, render_template, request, Response
import pandas as pd
import os, io, csv, sys, pickle, time
from werkzeug.utils import secure_filename

In [18]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")

ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [17]:
!git lfs install
!git clone https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

Updated git hooks.
Git LFS initialized.


Cloning into 'distilbert-base-uncased-finetuned-sst-2-english'...
fatal: unable to access 'https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/': OpenSSL SSL_connect: Connection was reset in connection to huggingface.co:443 


In [19]:
!pip install -r /content/character-network/requirements_sentiment_analysis.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/character-network/requirements_sentiment_analysis.txt'


In [3]:
with open(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\book_content.pkl', 'rb') as f:
    book_content = f.read()
with open(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\sentiment_lables.pkl', 'rb') as f:
    sentiment_lables = f.read()
with open(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\encoded_sentiment_labels.pkl', 'rb') as f:
    encoded_sentiment_labels = f.read()
with open(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\emotions_count.pkl', 'r') as f:
    emotions_count = f.read()

In [6]:
pd.read_pickle(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\emotions_count.pkl')

{'POSITIVE': 2530, 'NEGATIVE': 3896, 'NEUTRAL': 258}

In [4]:
emotions_count

'€\x03}q\x00(X\x08\x00\x00\x00POSITIVEq\x01Mâ\tX\x08\x00\x00\x00NEGATIVEq\x02M8\x0fX\x07\x00\x00\x00NEUTRALq\x03M\x02\x01u.'

In [ ]:
# sentiment analysis on sentences
# we use gpu for this task

def senti_analysis_transformers(sentences, plot=True):


    """
    given the fact that we have GPU on colab
    We empty the cache and initialize our sentiment analysis
    """
    print('Defining the sentiment of each sentence using a semantic model...')
    torch.cuda.empty_cache()

    sentiments_lists = []

    # we use small batches to prevent crashing
    for i in range(0, len(sentences), 50):
        if i%1000==0 : print('sentence', colored(f"{i}", 'blue'))
        sentiments_lists.append(classifier(sentences[i: i+50]))
    
    # list of list -> list
    all_sentences_sentiment = list(chain.from_iterable(sentiments_lists))

    labels, encoded_labels = decide_for_sentiment_label(all_sentences_sentiment)

    
    # count the labels
    
    emotions_count = dict(Counter(labels))
    print(colored('Emotions dominance: \n', 'blue'), emotions_count)
    
    if plot:plot_emotions(emotions_count)
    
    return labels, encoded_labels, emotions_count

sentiment_lables, encoded_sentiment_labels, emotions_count = senti_analysis_transformers(book_sents_cleaned, plot=False)
